Copyright 2016 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

--------------------------------------

This notebook is similar in functionality to [this python script](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/mnist_tflearn.py), and is used with [this README](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/02_README_mnist_tflearn.md).  It shows how to use TensorFlow's high-level apis, in `contrib.tflearn`, to easily build a classifier with multiple hidden layers.

First, do some imports and set some variables:

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import time

import numpy
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

DATA_DIR = "./the_hard_way/MNIST_data"

# read in data, downloading first as necessary
DATA_SETS = input_data.read_data_sets(DATA_DIR)

# comment out for less info during the training runs.
tf.logging.set_verbosity(tf.logging.INFO)

# define a utility function for generating a new directory in which to save 
# model information, so multiple training runs don't stomp on each other.
def getNewPath(base="/tmp/tfmodels/mnist_tflearn"):
    logpath = os.path.join(base, str(int(time.time())))
    print("Logging to {}".format(logpath))
    return logpath


Extracting ./the_hard_way/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./the_hard_way/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./the_hard_way/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./the_hard_way/MNIST_data/t10k-labels-idx1-ubyte.gz


We'll first define a function that adds a LinearClassifier and runs its `fit()` method, which will train the model. Note that we didn't need to explicitly define a model graph or a training loop ourselves.  

Once we've trained the model, we run the `evaluate()` method, which uses the trained model. To do this, it loads the most recent checkpointed model info available.  The model checkpoint(s) will be generated during the training process.

In [2]:
def define_and_run_linear_classifier(num_steps, logdir):
    """Run a linear classifier."""
    global DATA_SETS
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(
        DATA_SETS.train.images)
    classifier = tf.contrib.learn.LinearClassifier(
                feature_columns=feature_columns, 
                n_classes=10,
               model_dir=logdir
                )
    classifier.fit(DATA_SETS.train.images,
                   DATA_SETS.train.labels.astype(numpy.int64),
                   batch_size=100, steps=num_steps
                  )
    print("Finished training.")
    # Evaluate accuracy.
    accuracy_score = classifier.evaluate(
        DATA_SETS.test.images,
        DATA_SETS.test.labels.astype(numpy.int64))['accuracy']
    print('Linear Classifier Accuracy: {0:f}'.format(accuracy_score))

Next, add a function that defines a `DNNClassifier`, and runs its `fit()` method, which will train the model. Again note that we didn't need to explicitly define a model graph or a training loop ourselves.  

Then after we've trained the model, we run the classifier's `evaluate()` method, which uses the trained model. 

In [3]:
def define_and_run_dnn_classifier(num_steps, logdir, lr=0.1):
    """Run a DNN classifier."""
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(DATA_SETS.train.images)
    
    classifier = tf.contrib.learn.DNNClassifier(
        feature_columns=feature_columns, 
        n_classes=10,
        hidden_units=[128, 32],
        # After you've done a training run with optimizer learning rate 0.1,
        # change it to 0.5 and run the training again.  Use TensorBoard to take
        # a look at the difference.  You can see both runs by pointing it to the
        # parent model directory, which by default is:
        #   tensorboard --logdir=/tmp/tfmodels/mnist_tflearn
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
        )
    classifier.fit(DATA_SETS.train.images,
                   DATA_SETS.train.labels.astype(numpy.int64),
                   batch_size=100, max_steps=num_steps)
    print("Finished running the training via the fit() method")
    
    accuracy_score = classifier.evaluate(
        DATA_SETS.test.images,
        DATA_SETS.test.labels.astype(numpy.int64))['accuracy']
    print('DNN Classifier Accuracy: {0:f}'.format(accuracy_score))

Now we can call the functions that define and train our classifiers.  Let's start with the LinearClassifier, which won't be very accurate. 


In [4]:
print("Running Linear classifier ...")
define_and_run_linear_classifier(1000, getNewPath())

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'save_checkpoints_secs': 600, 'save_summary_steps': 100, 'tf_random_seed': None, '_job_name': None, 'cluster_spec': None, 'keep_checkpoint_max': 5, 'keep_checkpoint_every_n_hours': 10000, 'num_ps_replicas': 0, 'evaluation_master': '', 'task': 0, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, 'master': ''}
INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float32, shape=TensorShape([Dimension(None), Dimension(784)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float32, normalizer=None)
INFO:tensorflow:Create CheckpointSaverHook


Running Linear classifier ...
Logging to /tmp/tfmodels/mnist_tflearn/1481183342


INFO:tensorflow:loss = 2.30259, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tfmodels/mnist_tflearn/1481183342/model.ckpt.
INFO:tensorflow:loss = 0.391843, step = 101
INFO:tensorflow:loss = 0.304103, step = 201
INFO:tensorflow:loss = 0.380147, step = 301
INFO:tensorflow:loss = 0.429173, step = 401
INFO:tensorflow:loss = 0.33165, step = 501
INFO:tensorflow:loss = 0.175632, step = 601
INFO:tensorflow:loss = 0.282347, step = 701
INFO:tensorflow:loss = 0.259106, step = 801
INFO:tensorflow:loss = 0.292558, step = 901
INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tfmodels/mnist_tflearn/1481183342/model.ckpt.
INFO:tensorflow:Loss for final step: 0.240117.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float32, normalizer=None)
INFO:tensorflow:Restored model from /tmp/tfmodels/mnist_tflearn/1481183342
INFO:tensorflow:Eval steps [0,inf) for training step 1000.


Finished training.


INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 1000 step: accuracy = 0.9218, loss = 0.283391


Linear Classifier Accuracy: 0.921800


Now, let's run the DNN Classifier.  First, let's try it with a .1 learning rate.

In [5]:
print("Running DNN classifier with .1 learning rate...")
classifier = define_and_run_dnn_classifier(5000, getNewPath(), lr=0.1)

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'save_checkpoints_secs': 600, 'save_summary_steps': 100, 'tf_random_seed': None, '_job_name': None, 'cluster_spec': None, 'keep_checkpoint_max': 5, 'keep_checkpoint_every_n_hours': 10000, 'num_ps_replicas': 0, 'evaluation_master': '', 'task': 0, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, 'master': ''}
INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float32, shape=TensorShape([Dimension(None), Dimension(784)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float32, normalizer=None)


Running DNN classifier with .1 learning rate...
Logging to /tmp/tfmodels/mnist_tflearn/1481183417


Instructions for updating:
Please use tf.contrib.framework.load_variable instead
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 2.37413, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tfmodels/mnist_tflearn/1481183417/model.ckpt.
INFO:tensorflow:loss = 0.333397, step = 101
INFO:tensorflow:loss = 0.293119, step = 201
INFO:tensorflow:loss = 0.249066, step = 301
INFO:tensorflow:loss = 0.173243, step = 401
INFO:tensorflow:loss = 0.186846, step = 501
INFO:tensorflow:loss = 0.0889015, step = 601
INFO:tensorflow:loss = 0.0852823, step = 701
INFO:tensorflow:loss = 0.122428, step = 801
INFO:tensorflow:loss = 0.081551, step = 901
INFO:tensorflow:loss = 0.0973008, step = 1001
INFO:tensorflow:loss = 0.134592, step = 1101
INFO:tensorflow:loss = 0.0369817, step = 1201
INFO:tensorflow:loss = 0.0427609, step = 1301
INFO:tensorflow:loss = 0.0393555, step = 1401
INFO:tensorflow:loss = 0.0423736, step = 1501
INFO:tensorflow:loss = 0.0543201, step = 1601
INFO:tensorflow:los

Finished running the training via the fit() method


INFO:tensorflow:Restored model from /tmp/tfmodels/mnist_tflearn/1481183417
INFO:tensorflow:Eval steps [0,inf) for training step 5000.
INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 5000 step: accuracy = 0.9784, loss = 0.0700139


DNN Classifier Accuracy: 0.978400


Now, let's run it with a .5 learning rate.

In [6]:
print("Running DNN classifier with .1 learning rate...")
classifier = define_and_run_dnn_classifier(5000, getNewPath(), lr=0.5)

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
INFO:tensorflow:Using config: {'save_checkpoints_secs': 600, 'save_summary_steps': 100, 'tf_random_seed': None, '_job_name': None, 'cluster_spec': None, 'keep_checkpoint_max': 5, 'keep_checkpoint_every_n_hours': 10000, 'num_ps_replicas': 0, 'evaluation_master': '', 'task': 0, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, 'master': ''}
INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float32, shape=TensorShape([Dimension(None), Dimension(784)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(None)]), is_sparse=False)
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float32, normalizer=None)


Running DNN classifier with .1 learning rate...
Logging to /tmp/tfmodels/mnist_tflearn/1481183497


Instructions for updating:
Please use tf.contrib.framework.load_variable instead
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 2.33871, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tfmodels/mnist_tflearn/1481183497/model.ckpt.
INFO:tensorflow:loss = 1.90681, step = 101
INFO:tensorflow:loss = 1.58743, step = 201
INFO:tensorflow:loss = 1.38492, step = 301
INFO:tensorflow:loss = 0.98334, step = 401
INFO:tensorflow:loss = 0.824214, step = 501
INFO:tensorflow:loss = 0.380703, step = 601
INFO:tensorflow:loss = 0.46079, step = 701
INFO:tensorflow:loss = 0.300015, step = 801
INFO:tensorflow:loss = 0.230406, step = 901
INFO:tensorflow:loss = 0.339944, step = 1001
INFO:tensorflow:loss = 0.398754, step = 1101
INFO:tensorflow:loss = 0.319462, step = 1201
INFO:tensorflow:loss = 0.110461, step = 1301
INFO:tensorflow:loss = 0.252838, step = 1401
INFO:tensorflow:loss = 0.176851, step = 1501
INFO:tensorflow:loss = 0.18334, step = 1601
INFO:tensorflow:loss = 0.705764, 

Finished running the training via the fit() method


INFO:tensorflow:Restored model from /tmp/tfmodels/mnist_tflearn/1481183497
INFO:tensorflow:Eval steps [0,inf) for training step 5000.
INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 5000 step: accuracy = 0.9584, loss = 0.146584


DNN Classifier Accuracy: 0.958400


To compare your results, start up TensorBoard as follows in a new terminal window. (If you get a 'not found' error, make sure you've activated your virtual environment in that new window):

```sh
$ tensorboard --logdir=/tmp/tfmodels/mnist_tflearn
```